In [1]:
import pyprind
import pandas as pd
import os

In [11]:
basepath = '../../../../moviedata'

In [12]:
labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']

/tmp/ipykernel_20813/1067533257.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([[txt, labels[l]]], ignore_index=True)


In [13]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('./resources/movie_data.csv', index = False, encoding = 'utf-8')

In [14]:
df = pd.read_csv('./resources/movie_data.csv', encoding='utf-8')
df.head(5)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [15]:
df.shape

(50000, 2)

In [19]:
"""

텍스트나 단어 같은 범주형 데이터를 머신 러닝 알고리즘에 주입하기 전에 수치형 형태로 변환
텍스트를 수치 특성 벡터로 표현 BoW(Bag-of-Word)

1. 전체 문서에 대해 고유한 토큰, 예를 들어 단어로 이루어진 어휘 사전을 만듦

2. 특정 문서에 각 단어가 얼마나 자주 등장하는지 헤아려 문서의 특성 벡터를 만듦

각 문서에 있는 고유한 단어는 BoW 어휘 사전에 있는 모든 단어의 일부분에 지나지 않으므로 특성 벡터는 대부분 0으로 채워짐.

그래서 이 특성 벡터를 희소하다고 함.

"""

# 단어를 특성 벡터로 변환

from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()       # ngram_range = (2,2) : 2-그램 표현
docs = np.array([
    'The sun is shining',
    'The weather is sweet',
    'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)
count.vocabulary_               # 어휘 사전 딕셔너리{단어 : 인덱스}

{'the': 6,
 'sun': 4,
 'is': 1,
 'shining': 3,
 'weather': 8,
 'sweet': 5,
 'and': 0,
 'one': 2,
 'two': 7}

In [18]:
bag.toarray()   # 3행 (문장 3개) , 9열 (단어 9개)
                # 특성 벡터의 이런 값들을 단어 빈도라고도 함
                # 문서 d에 등장한 단어 t의 횟수를 tf(t, d)와 같이 씀

array([[0, 1, 0, 1, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 1, 0, 0, 1],
       [2, 3, 2, 1, 1, 1, 2, 1, 1, 0]])

In [23]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

```
tf-idf(term frequency-inverse documnet frequency)를 사용하여 단어 적합성 평가
특성 벡터에서 자주 등장하는 단어의 가중치를 낮추는 기법
tf-idf는 단어 빈도(tf)와 역문서 빈도(inverse documetn frequency)의 곱

tf-idf(t ,d) = tf(t, d) x idf(t, d)
싸이킷런에서 계산하는 idf : idf(t, d) = log((1 + n) / (1 + df(d, t)))
싸이킷런에서 계산하는 tf-idf : tf-idf(t, d) = tf(t, d) x (idf(t, d) + 1)

idf(t, d) = log(n_d / (1 + df(d, t)))
n_d : 전체 문서 개수
df(d, t)는 단어 t가 포함된 문서 d의 개수

사이킷런 라이브러리에는 CountVectorizer 클래스에서 만든 단어 빈도를 입력받아 tf-idf로 변환하는 TfidfTransformer 클래스가 구현되어 있음
```

In [26]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(norm='l2', use_idf=True, smooth_idf=True)
                        
np.set_printoptions(precision = 2)
count.vocabulary_
tfidf.fit_transform(count.fit_transform(docs)).toarray()

{'the': 6,
 'sun': 4,
 'is': 1,
 'shining': 3,
 'weather': 8,
 'sweet': 5,
 'and': 0,
 'one': 2,
 'two': 7}

array([[0.  , 0.43, 0.  , 0.56, 0.56, 0.  , 0.43, 0.  , 0.  ],
       [0.  , 0.43, 0.  , 0.  , 0.  , 0.56, 0.43, 0.  , 0.56],
       [0.5 , 0.45, 0.5 , 0.19, 0.19, 0.19, 0.3 , 0.25, 0.19]])

### 텍스트 데이터 정제

1. 불필요한 문자를 삭제하여 텍스트 데이터를 정제

2. 어떻게 텍스트 문서를 낱개의 토큰으로 나눌지 생각
    - 문서를 토큰화하는 한 가지 방법은 공백 문자를 기준으로 개별 단어로 나누는 것

In [28]:
df.head(5)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [29]:
df.loc[0, 'review'][-50:]
# HTML은 물론 구두점과 글자가 아닌 문자까지 포함되어 있다.
# 구두점은 특정 NLP 문제에서 쓸모 있는 추가 정보가 될 수 있다.

'is seven.<br /><br />Title (Brazil): Not Available'

In [30]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)      # 영화 리뷰에서 HTML 태그 삭제 (고급 도구를 사용하고 싶다면 파이썬의 HTML 파서 모듈 사용)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)    # 이모티콘
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))  # 텍스트 소문자, [\W]+를 사용하여 텍스트에서 단어가 아닌 문자를 모두 제거
    return text

In [31]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [32]:
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

In [33]:
# 정제된 텍스트 데이터를 반복해서 사용하기 때문에 데이터프레임에 있는 모든 영화 리뷰에 preprocessor 함수를 적용
df['review'] = df['review'].apply(preprocessor)

In [34]:
def tokenizer(text):
    return text.split()
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [35]:
# 어간 추출
# 여러 가지 형태를 갖는 단어를 같은 어간으로 매핑
# 포터 어간 추출기 알고리즘
# 가장 오래되고 간단한 어간 추출 알고리즘
# nltk 패키지
# http://www.nltk.org/book/

# 인기 있는 다른 어간 추출 : 스노우볼 어간 추출기, 랭커스터 어간 추출기
# 표제어 추출 기법은 표제어라는 (각 단어의 표준 형태를 얻는 것이 목적)
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]
tokenizer_porter('runner like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [36]:
# 불용어 제거
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jngmk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [37]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

In [38]:
# BoW 모델을 기반으로 영화 리뷰를 긍정과 부정 리뷰로 분류하는 로지스틱 회귀 모델을 훈련
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [42]:
# GridSearchCV 객체에서 5-겹 계층별 교차 검증을 사용하여 로지스틱 회귀 모델에 대한 최적의 매개변수 조합 찾기
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [43]:
tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)
param_grid = [{
                'vect__ngram_range' : [(1, 1)],
                'vect__stop_words': [stop, None],
                'vect__tokenizer': [tokenizer, tokenizer_porter],
                'clf__penalty': ['l1', 'l2'],
                'clf__C': [1.0, 10.0, 100.0]
                },
            {
                'vect__ngram_range': [(1, 1)],
                'vect__stop_words': [stop, None],
                'vect__tokenizer': [tokenizer, tokenizer_porter],
                'vect__use_idf': [False], 'vect__norm': [None],         # 단어 빈도를 사용하여 모델을 훈련
                'clf__penalty': ['l1', 'l2'],                           # l1, l2 규제 적용
                'clf__C': [1.0, 10.0, 100.0]                            # 규제 매개변수 C 규제 강도를 비교
            }]

In [46]:
lr_tfidf = Pipeline([('vect', tfidf), ('clf', LogisticRegression(solver='liblinear', random_state=0))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=1)
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


/home/jngmk/anaconda3/envs/venv2/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  warnings.warn(
/home/jngmk/anaconda3/envs/venv2/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  warnings.warn(
/home/jngmk/anaconda3/envs/venv2/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your sto

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0,
                                                           solver='liblinear'))]),
             n_jobs=1,
             param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've"...
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
 

In [47]:
print('최적의 매개변수 조합: %s' % gs_lr_tfidf.best_params_)
# 포터 어간 추출을 하지 않는 tokenizer 함수와 tf-idf를 사용하고 불용어 제거는 사용하지 않는 경우 최상의 그리드 서치 결과

최적의 매개변수 조합: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x7f6e3a3bf0d0>}


In [48]:
# 그리드 서치로 찾은 최상의 모델을 사용하여 훈련 데이터셋에 대한 모델의 5-겹 교차 검증 정확도와 테스트 데이터셋에 대한 분류 정확도
print('CV 정확도 : %.3f' % gs_lr_tfidf.best_score_)
clf = gs_lr_tfidf.best_estimator_
print('테스트 정확도 : %.3f' % clf.score(X_test, y_test))

CV 정확도 : 0.897
테스트 정확도 : 0.899
